In [1]:
from bs4 import BeautifulSoup  
import urllib 

## Make an request to website  

In [2]:
def get_soup(url):
    """Makes an url request and return url as BeautifulSoup"""
    html_page = urllib.request.urlopen(url)
    soup = BeautifulSoup(html_page)
    return soup

## From the Search URL, Get the number of pages to get all results

In [4]:
url = 'https://www.collegetuitioncompare.com/search/?program=51.0806'
soup = get_soup(url)

result_count = soup.find("div", {"class": "resultdesc count"})
print(result_count)
result_count = int(result_count.text.split(" ")[0])
if result_count % 10 >0:
    pages = int(result_count/10) + 1 
else: 
    pages = int(result_count/10) 
print(pages)

<div class="resultdesc count">381 matched schools</div>
39


## Get the name and links of all schools from all the result pages.

In [13]:
schools = dict()
count= 1 
pages= 2  # Hard Coded for only 1 page. Remove to run all
for i in range(1,pages+1):
    url = 'https://www.collegetuitioncompare.com/search/?program=51.0806&page='+str(i)
    soup = get_soup(url)
    results = soup.find_all("div", {"class": "searchedschools"})
    for sch in results:
        schools[count] =dict()
        name = sch.find("h3").find("a").string
        link = sch.find("h3").find("a")['href']
        schools[count]["Name"] = name 
        schools[count]["link"] = 'https:'+link  
        count = count + 1

In [20]:
schools[1]["Name"]

'South University-Montgomery'

## Go to the school page and get additional info 

In [15]:
for key, value in schools.items():
    url = value['link']
    soup = get_soup(url) 
    soup = soup.find("div",{"class":'withbread'})  
    tables = soup.findAll( "table" ) 
    for table in tables:  
        thead = table.find("thead") 
        if thead is not None :
            theads = thead.findAll('th') 
            for row in table.findAll("tr"): 
                th = row.find('th')
                td = row.findAll('td') 
                if len(td)>0:
                    for i in range(0,len(theads)-1):
                        value[str(th.string)+"_"+str(theads[i+1].string)]= td[i].string
        else:
            for row in table.findAll("tr"): 
                th = row.find('th').string  
                try:
                    td = row.find('td').string 
                    if td is None:
                        td = (row.find('td').find('a')['href'])
                except:
                    td=None
                if th is not None:
                    value[str(th)] = td

In [12]:
schools

{1: {'Name': 'South University-Montgomery',
  'link': 'https://www.collegetuitioncompare.com/edu/101116/south-university-montgomery',
  'Address': '5355 Vaughn Rd, Montgomery, AL 36116',
  'Phone': '334-395-8800',
  'WEB': 'https://www.southuniversity.edu/montgomery#location=Montgomery,%20AL',
  'Admission': 'https://www.southuniversity.edu/admissions',
  'Online Application': 'https://www.southuniversity.edu/admissions/admission-info/applying',
  'Financial Aid': 'https://www.southuniversity.edu/admissions/financial-aid',
  'Net Price Calculator': 'https://tcc.noellevitz.com/edmc/South Net Price Calculator',
  'Veterans Tuition': 'https://www.southuniversity.edu/Admissions/Military/military-faqs',
  'Disability Service': 'https://content.edmc.edu/assets/documents/su/disability-services-su.pdf',
  'School Type': 'Private (for-profit), 4 or more years',
  'Carnegie Category': 'Baccalaureate College - Diverse Fields',
  'Highest Degree Offered': "Doctor's degree -  professional practice"

## Convert the dictionary to Panda Dataframe

In [21]:
import pandas as pd 

In [22]:
schools_df= pd.DataFrame.from_dict(schools, orient='index') 
#schools_df.to_excel("College Tuition Compare 51.0806 v1.xlsx", index = False)

In [23]:
schools_df

,Name,link,Address,Phone,WEB,Admission,Online Application,Financial Aid,Net Price Calculator,Veterans Tuition,...,Dormitory Capacity,Athletic,Undergraduate_In-State,Undergraduate_Out-of-State,Transfer-out Rate,Historical Black School,Graduate_In-State,Graduate_Out-of-State,Admission Stats »,Test Scores »
1,South University-Montgomery,https://www.collegetuitioncompare.com/edu/1011...,"5355 Vaughn Rd, Montgomery, AL 36116",334-395-8800,https://www.southuniversity.edu/montgomery#loc...,https://www.southuniversity.edu/admissions,https://www.southuniversity.edu/admissions/adm...,https://www.southuniversity.edu/admissions/fin...,https://tcc.noellevitz.com/edmc/South Net Pric...,https://www.southuniversity.edu/Admissions/Mil...,...,No Campus Housing Facilities Provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,George C Wallace Community College-Dothan,https://www.collegetuitioncompare.com/edu/1012...,"1141 Wallace Drive, Dothan, AL 36303-9234",334-983-3521,https://www.wallace.edu,https://www.wallace.edu/admissions.aspx,https://www.wallace.edu/admissions/online_appl...,https://www.wallace.edu/financial_aid.aspx,https://www.wallace.edu/net_price_calculator.aspx,https://www.wallace.edu/financial_aid/veterans...,...,No Campus Housing Facilities Provided,https://www.wallace.edu/about_wcc/consumer_inf...,"$4,801","$8,792",18%,NaN,NaN,NaN,NaN,NaN
3,George C Wallace State Community College-Hance...,https://www.collegetuitioncompare.com/edu/1012...,"801 Main St NW, Hanceville, AL 35077-2000",256-352-8000,https://www.wallacestate.edu,https://www.wallacestate.edu/Admissions,https://www.wallacestate.edu/Admissions/Apply-...,https://www.wallacestate.edu/Financial-Aid,https://www.wallacestate.edu/Financial-Aid/Net...,https://www.wallacestate.edu/Financial-Aid/Vet...,...,180,https://www.wallacestate.edu/consumer-information,"$4,861","$8,852",12%,NaN,NaN,NaN,NaN,NaN
4,Jefferson State Community College,https://www.collegetuitioncompare.com/edu/1015...,"2601 Carson Rd, Birmingham, AL 35215-3098",205-853-1200,https://www.jeffersonstate.edu/,https://www.jeffersonstate.edu/become-a-student/,https://www.jeffersonstate.edu/become-a-studen...,https://www.jeffersonstate.edu/financial-aid/,https://www.jeffersonstate.edu/financial-aid/n...,https://www.jeffersonstate.edu/financial/veter...,...,No Campus Housing Facilities Provided,NaN,"$4,921","$8,912",26%,NaN,NaN,NaN,NaN,NaN
5,John C Calhoun State Community College,https://www.collegetuitioncompare.com/edu/1015...,"6250 U S Highway 31 N, Tanner, AL 35671",256-306-2500,https://calhoun.edu/,https://calhoun.edu/admissions,https://banssb.calhoun.edu/PROD/bwskalog.P_Dis...,https://calhoun.edu/student-resources/financia...,https://calhoun.studentaidcalculator.com/surve...,https://calhoun.edu/student-resources/financia...,...,No Campus Housing Facilities Provided,https://calhoun.edu/student-resources/consumer...,"$4,941","$8,932",23%,NaN,NaN,NaN,NaN,NaN
6,Bishop State Community College,https://www.collegetuitioncompare.com/edu/1020...,"351 North Broad Street, Mobile, AL 36603-5898",251-405-7000,https://www.bishop.edu,https://www.bishop.edu/admissions/,https://www.bishop.edu/admissions-records-efor...,https://www.bishop.edu/financial-affairs/finan...,https://www.bishop.edu/calculator/npcalc.htm,https://www.bishop.edu/financial-affairs/finan...,...,No Campus Housing Facilities Provided,NaN,"$4,740","$8,610",16%,Yes,NaN,NaN,NaN,NaN
7,University of Alaska Anchorage,https://www.collegetuitioncompare.com/edu/1025...,"3211 Providence Drive, Anchorage, AK 99508",907-786-1800,https://www.uaa.alaska.edu,https://www.uaa.alaska.edu/admissions/,https://uaonline.alaska.edu/,https://www.uaa.alaska.edu/financialaid/,https://www.uaa.alaska.edu/financialaid/custom...,NaN,...,977,https://fs.ncaa.org/DOCS/newmedia/public/rates...,"$7,653","$31,852",18%,NaN,"$15,640","$31,007",83% Acceptance Rate / 45% Yield,NaN
8,Carrington College-Mesa,https://www.collegetuitioncompare.com/edu/1039...,"1001 W. Southern Ave., Ste. 130, Mesa, AZ 85210",480-2